In [ ]:
#IMPORTING AND SHOWCASING 

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk import pos_tag
import copy
reference = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']] #deve essere cosi 
candidate = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', 'from', 'space']
chencherry = SmoothingFunction() #nel caso in cui non ci sono parole uguali, almeno non da 0 ma è less-harsh, pià utile per le comparison

#Explanation of n-grams (difference between bleu-1 (not used) vs bleu-2 vs bleu-3 vs bleu-4 vs bleu-5)
'''
In practice, however, using individual words as the unit of comparison is not optimal. 
Instead, BLEU computes the same modified precision metric using n-grams. 
The length which has the "highest correlation with monolingual human judgements"[5] 
was found to be four. The unigram scores are found to account for the adequacy of the translation,
how much information is retained. The longer n-gram scores account for the fluency of the translation,
or to what extent it reads like "good English".
https://en.wikipedia.org/wiki/BLEU
'''

#bleu-2
weights = (1./2.,1./2.,1./2.)
score = sentence_bleu(reference, candidate, weights,smoothing_function = chencherry.method1)

#bleu-3
weights = (1./3.,1./3.,1./3.)
score = sentence_bleu(reference, candidate, weights,smoothing_function = chencherry.method1)

#bleu-4 is standard, you could omit the weights
weights = (1./4.,1./4.,1./4.,1./4.)
score = sentence_bleu(reference, candidate, weights, smoothing_function = chencherry.method1)

#bleu-5
weights = (1./5., 1./5., 1./5., 1./5., 1./5.)
score = sentence_bleu(reference, candidate, weights,smoothing_function = chencherry.method1)

In [ ]:
### BLEU SCORE FUNCTION AND FUNCTIONALITY TEST ###

In [ ]:
def bleu_score(references, model_output, k):
    '''
    references is the y_true, list of string
    model output, list of string
    k is the number of n-grams you wanto to consider, from 2 to 5 is acceptable
    
    the "right" sentence for model_output[i] is references[i].
    '''
    if len(references) != len(model_output):
        print("Error, references and model_output must be same length")
        return -1
    if k < 2 or k > 5:
        print("Error, k must be between 2 and 5")
        return -2
    
    N = len(references)
    sum_scores = 0
    chencherry = SmoothingFunction()
    
    if k == 2:
        weights = (1./2.,1./2.)
    elif k == 3:
        weights = (1./3.,1./3.,1./3.)
    elif k == 4:
        weights = (1./4.,1./4.,1./4.,1./4.)
    else:
        #k==5
        weights = (1./5.,1./5.,1./5.,1./5.,1./5.)
    
    
    for i in range(N):
        ref = references[i]
        out = model_output[i]
        
        ref_words = str.split(ref)
        out_words = str.split(out)
        
        score_i = sentence_bleu([ref_words],out_words,weights,smoothing_function = chencherry.method1)
        
        sum_scores += score_i
        
    return sum_scores/N

In [ ]:
## TESTING BLEU SCORE ## 
references = ["The cat is on the table", "The red fox was looking at the sheep"]
model_output = ["The cat is on the table", "The blue fox is looking at the moon"]
bleu_score(references, model_output, 2)
bleu_score(references,model_output,3)
bleu_score(references,model_output,4)
bleu_score(references,model_output,5)

In [ ]:
### SELF-BLEU FUNCTION ### 

def self_bleu_score(model_output, k):
    '''
    model_output has list of strings
    k is the n-grams number n as before
    
    '''
    
    N = len(model_output)
    
    sum_scores = 0
    
    chencherry = SmoothingFunction()
    
    if k == 2:
        weights = (1./2.,1./2.)
    elif k == 3:
        weights = (1./3.,1./3.,1./3.)
    elif k == 4:
        weights = (1./4.,1./4.,1./4.,1./4.)
    else:
        #k==5
        weights = (1./5.,1./5.,1./5.,1./5.,1./5.)
    
    outputs = []
    for el in model_output:
        outputs.append(str.split(el))
    
    for i in range(N):
        out_words_copied = outputs.copy()
        #remove the i-esim string
        candidate = out_words_copied.pop(i)
        
        score_i = sentence_bleu(out_words_copied,candidate,weights,smoothing_function = chencherry.method1)
        
        sum_scores += score_i
    
    
    return sum_scores/N

In [ ]:
## TESTING SELF-BLEU SCORE ## 
references = ["The cat is on the table", "The red fox was looking at the sheep", "The kid is talking"]
model_output = ["The cat is on the table", "The blue fox is looking at the moon", "The kid was sleeping"]
self_bleu_score(model_output,2)
self_bleu_score(model_output,3)
self_bleu_score(model_output,4)
self_bleu_score(model_output,5)

In [ ]:
### POS-BLEU FUNCTION ###
def pos_bleu_score(references, model_output, k):
    '''
    references is the y_true, list of string
    model output, list of string
    k is the number of n-grams you wanto to consider, from 2 to 5 is acceptable
    
    the "right" sentence for model_output[i] is references[i].
    
    This is the same as bleu_score, with the only difference that now the senteces 
    (both reference and model_outputs) are changed as Part of speech tags
    '''
    if len(references) != len(model_output):
        print("Error, references and model_output must be same length")
        return -1
    if k < 2 or k > 5:
        print("Error, k must be between 2 and 5")
        return -2
    
    N = len(references)
    sum_scores = 0
    chencherry = SmoothingFunction()
    
    if k == 2:
        weights = (1./2.,1./2.)
    elif k == 3:
        weights = (1./3.,1./3.,1./3.)
    elif k == 4:
        weights = (1./4.,1./4.,1./4.,1./4.)
    else:
        #k==5
        weights = (1./5.,1./5.,1./5.,1./5.,1./5.)
    
    
    for i in range(N):
        ref = references[i]
        out = model_output[i]
        ref_words = word_tokenize(ref)
        out_words = word_tokenize(out)
        
        ref_pos = pos_tag(ref_words)
        out_pos = pos_tag(out_words)
        
        ref_words = []
        for i in range(len(ref_pos)):
            ref_words.append(ref_pos[i][1])
        
        out_words = []
        for i in range(len(out_pos)):
            out_words.append(out_pos[i][1])
        
        score_i = sentence_bleu([ref_words],out_words,weights,smoothing_function = chencherry.method1)
        
        sum_scores += score_i
        
    return sum_scores/N

In [ ]:
## TESTING POS-BLEU SCORE ## 
references = ["The cat is on the table", "The red fox was looking at the sheep"]
model_output = ["The cat is on the table", "The blue fox is looking at the moon"]
pos_bleu_score(references, model_output,2)
pos_bleu_score(references,model_output,3)
pos_bleu_score(references,model_output,4)
pos_bleu_score(references,model_output,5)